In [ ]:
import os
import cv2
import pandas as pd
import numpy as np

In [ ]:
# Paths
path = 'DeepRN'
path_exams = "D:\\Projects\\Datasets"

path_data = "{}\\data.csv".format(path_exams)

path_log = 'log_result.log'

In [ ]:
# Params
qt_frames_instantly_before = 8
qt_frames_instantly_after = 8

qt_frames_median_get_size = 5

qt_values_median_variation = 3

In [ ]:
# Open a data

data = pd.read_csv(path_data)
data.head()

In [ ]:
def get_fps_video(video):
    exam_path = "{}\\Exams\\{}.mp4".format(path_exams, video)
    exam = cv2.VideoCapture(exam_path)
    return exam.get(cv2.CAP_PROP_FPS)

In [ ]:
def load_label_data(video):
    label_path = "{}\\{}_label.csv".format(path, video)
    return pd.read_csv(label_path)

In [ ]:
def get_list_stimulus(stimulus):
    aux = stimulus.replace('[', '').replace(']', '')
    list_stimuls = aux.split('}, {')
    
    if len(list_stimuls) == 0:
        raise Exception('Falha por nao ter estimulos')
        
    return list_stimuls

In [ ]:
def describe_stimul(stimul):
    stimul = stimul.replace('{', '').replace('}', '')

    # Get a color stimul
    stimul_color = stimul[stimul.find(':')+2:stimul.find(",")]
    stimul_color = stimul_color.replace("'", '')
    stimul = stimul[stimul.find(",")+2:]

    # Get a init frame stimul
    init_stimul = int(stimul[stimul.find(':')+2:stimul.find(",")])
    stimul = stimul[stimul.find(",")+2:]

    # Get a end frame stimul
    end_stimul = int(stimul[stimul.find(':')+2:len(stimul)])
    
    return stimul_color, init_stimul, end_stimul

In [ ]:
def get_size_stimul(label, number_frame):
    range_min = number_frame - qt_frames_median_get_size
    range_min = range_min if range_min > 0 else 0
    
    range_max = number_frame + qt_frames_median_get_size + 1
    range_max = range_max if range_max < label.shape[0] else label.shape[0]-1
    
    list_sizes = np.array([])
    for i in range(range_min, range_max):
        list_sizes = np.append(list_sizes, label['radius'][i])    

    list_sizes.sort()
    list_sizes = list_sizes[1:len(list_sizes)-1]
    
    return np.mean(list_sizes)

In [ ]:
def get_sizes_instantly_after_before(label, init_stimul, end_stimul):
    frame_instantly_before = init_stimul - qt_frames_instantly_before
    frame_instantly_after = end_stimul + qt_frames_instantly_after

    size_instantly_before = get_size_stimul(label, frame_instantly_before)
    size_instantly_after = get_size_stimul(label, frame_instantly_after)    
    
    return size_instantly_before, size_instantly_after

In [ ]:
def normalize(x):
    x = np.asarray(x)
    y = (x - x.min()) / (np.ptp(x))
    return y * x.max()

In [ ]:
def save_log(information):
    with open(path_log, 'a') as file_writer:
        file_writer.write('{}\n'.format(information))

In [ ]:
def get_max_variation(label, init_range, end_range):
    list_sizes = np.array([])
    
    init_range = init_range if init_range > 0 else 0
    end_range = end_range if end_range < label.shape[0]-1 else label.shape[0]-1
    
    save_log('func get_max_variation init_range: {} end_range: {}'.format(init_range, end_range))
    
    for i in range(init_range, end_range-1):
        list_sizes = np.append(list_sizes, label['radius'][i])
    
    list_sizes.sort()
    
    max_variation = 0
    if len(list_sizes) > (qt_values_median_variation * 2):        
        for i in range(qt_values_median_variation):
            max_variation = max_variation + (list_sizes[(len(list_sizes)-1)-i] - list_sizes[i])

        max_variation = max_variation / qt_values_median_variation
    
    return max_variation

In [ ]:
data_final = pd.DataFrame({'patient': [], 'classify': [], 'video': [], 'param': [], 'stimul': [],
                           '3_seconds_before': [],
                           'max_variation_3_init': [],
                           'size_instantly_before_stimul': [],
                           'size_instantly_after_stimul': [],
                           'max_variation_end_3': [],
                           '3_seconds_after': [],
                           'max_variation_3_5': [],
                           '5_seconds_after': [],
                           'max_variation_5_6': [],
                           '6_seconds_after': [],
                           'max_variation_6_10': [],
                           '10_seconds_after': [],
                           'max_variation_10_15': [],
                           '15_seconds_after': []})

for index, values in data.iterrows():    
    # Get a patient informations
    video = values['video']    
    patient = str(values['patient'])
    classify = str(values['classify'])
    param = str(values['param'])
    stimulus = str(values['stimulus'])
    
    # Get a FPS Exam
    fps = get_fps_video(video)
    
    # Get a labels of video
    label = load_label_data(video)

    if label.shape[0] <= 0:
        continue
    
    # Get a list of stimulus
    list_stimuls = get_list_stimulus(stimulus)    
    
    save_log('Exam: {}'.format(video))
    for i in range(0, len(list_stimuls)):
        # get information stimul
        stimul_color, init_stimul, end_stimul = describe_stimul(list_stimuls[i])
        save_log('stimul_color: {}, init_stimul: {}, end_stimul: {}'.format(stimul_color, init_stimul, end_stimul))
        
        # get size instantly before and after stimul
        size_instantly_before, size_instantly_after = get_sizes_instantly_after_before(label, init_stimul, end_stimul)
        
        # get sizes before and after stimul
        frame_3_seconds_before = init_stimul - int(fps * 3)
        
        frame_3_seconds_after = end_stimul + int(fps * 3)
        frame_5_seconds_after =  end_stimul + int(fps * 5)
        frame_6_seconds_after = end_stimul + int(fps * 6)
        frame_10_seconds_after = end_stimul + int(fps * 10)
        frame_15_seconds_after = end_stimul + int(fps * 15)

        save_log("frame_3_before: {}, frame_3_after: {}, frame_5_after: {}, frame_6_after: {} frame_10_after: {}, frame_15_ater: {}".format(frame_3_seconds_before, frame_3_seconds_after, frame_5_seconds_after, frame_6_seconds_after, frame_10_seconds_after, frame_15_seconds_after))
        
        # valid number frames and get size stimuls
        size_3_seconds_before = get_size_stimul(label, frame_3_seconds_before)

        max_variation_3_init = get_max_variation(label, frame_3_seconds_before, init_stimul)
                
        if i < len(list_stimuls)-1:
            _, limit_max, _ = describe_stimul(list_stimuls[i+1])            
        else:
            limit_max = label.shape[0]
            
        size_3_seconds_after = get_size_stimul(
            label, frame_3_seconds_after) if frame_3_seconds_after < limit_max else np.NaN
        
        max_variation_end_3 = get_max_variation(label, end_stimul, frame_3_seconds_after)
        

        size_5_seconds_after = get_size_stimul(
            label, frame_5_seconds_after) if frame_5_seconds_after < limit_max else np.NaN
        
        max_variation_3_5 = get_max_variation(label, frame_3_seconds_after, frame_5_seconds_after)
        

        size_6_seconds_after = get_size_stimul(
            label, frame_6_seconds_after) if frame_6_seconds_after < limit_max else np.NaN
        
        max_variation_5_6 = get_max_variation(label, frame_5_seconds_after, frame_6_seconds_after)
        

        size_10_seconds_after = get_size_stimul(
            label, frame_10_seconds_after) if frame_10_seconds_after < limit_max else np.NaN
        
        max_variation_6_10 = get_max_variation(label, frame_6_seconds_after, frame_10_seconds_after)
        

        size_15_seconds_after = get_size_stimul(
            label, frame_15_seconds_after) if frame_15_seconds_after < limit_max else np.NaN
        
        max_variation_10_15 = get_max_variation(label, frame_10_seconds_after, frame_15_seconds_after)
        
        # Add register final data
        data_final = data_final.append(pd.DataFrame({'patient': [patient], 'classify': [classify], 'video': [video],
                                                     'param': [param], 'stimul': [stimul_color],
                                                     '3_seconds_before': [size_3_seconds_before],
                                                     'max_variation_3_init': [max_variation_3_init],
                                                     'size_instantly_before_stimul': [size_instantly_before],
                                                     'size_instantly_after_stimul': [size_instantly_after],
                                                     'max_variation_end_3': [max_variation_end_3],
                                                     '3_seconds_after': [size_3_seconds_after],
                                                     'max_variation_3_5': [max_variation_3_5],
                                                     '5_seconds_after': [size_5_seconds_after],
                                                     'max_variation_5_6': [max_variation_5_6],
                                                     '6_seconds_after': [size_6_seconds_after],
                                                     'max_variation_6_10': [max_variation_6_10],
                                                     '10_seconds_after': [size_10_seconds_after],
                                                     'max_variation_10_15': [max_variation_10_15],
                                                     '15_seconds_after': [size_15_seconds_after]}))

print(data_final)

In [ ]:
data_final.to_csv('data_final.csv', index=False)